In [ ]:
from flygym import Camera, Simulation
import cv2
import numpy as np
from tqdm import trange
import flygym as flygym
from flygym.arena import FlatTerrain
from flygym.preprogrammed import all_leg_dofs
from flygym.vision import save_video_with_vision_insets
from scripts.utils import plot_chasing

In [ ]:
from scripts.chasing_fly import ChasingFly
from scripts.hybrid_turning_fly import HybridTurningFly
from scripts.colored_terrain import FlatTerrainColored